In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, sys
from collections import defaultdict
from matplotlib.backends import backend_pdf

from scipy.optimize import curve_fit

In [ ]:
def poly( v, a, k ):
    return a*np.power(v,k)

def poly_derivative ( v, a, k):
    return a*k*np.power(v, k-1)

def ipoly( g, a, ea, k, ek ): 
    
    v0 = np.power(g/a, 1./k)
    
    dvda = -(1./k)*np.power(0.5, 1./k)*np.power(1./a, 1./(k+1))
    dvdk = -(1./k**2)*np.power(0.5, 1./k)*np.power( (1./a)*(np.log(1./a)-np.log(0.5)), 1./k )
    
    ev0 = np.sqrt( (dvda*ea)**2 + (dvdk*ek)**2 )
    
    return v0, ev0

def dofit( data_x, data_y, error_y, g_target=0.4 ):
    """
    Args: 
        g_target: target gain in units of 10^7
    Returns: 
        ..
        ..
    """
    
    btrials=[5,6,7,7.5,8]
    trials=0
    b=btrials[0]
    
    params=[]
    pcov=[]
    
    repeatfit=True
    
    while repeatfit:
        try:
            #print("Try fit using b: %.2f" % b)
            params, pcov = curve_fit(poly, data_x, data_y, sigma=error_y, absolute_sigma=True, p0=[1e-13, b])
            repeatfit=False
        except:
            trials=trials+1
            if trials<len(btrials):
                b=btrials[trials]
                #print("Repeat fit using b: %.2f" % b)
            else: 
                #print("Finished the amount of trials available for fit")
                raise 
        else: 
            repeatfit=False
            
            
    stderr = np.sqrt(np.diag(pcov))
    
    xint = np.linspace( np.min(data_x)-10, np.max(data_x)+10, 1000 )
    yint = poly(xint, params[0], params[1])
    
    vnom, evnom= ipoly( g_target, params[0], stderr[0] , params[1], stderr[1] )
    
    #print(params, np.sqrt(np.diag(pcov)), vnom, evnom)
    
    return params, np.sqrt(np.diag(pcov)), xint, yint, vnom, evnom


def plotone( ch, pmtid, v, g, eg ):
    fig, ax = plt.subplots(1,1, figsize=(10, 4.8))

    print( ch, pmtid, v, g)
    
    kargs={'marker':'o', 'lw':0.0, 'elinewidth':2.0}

    out=ax.errorbar( x=v, y=g, yerr=eg, label="Channel ID: {} (PMT {})".format(ch, pmtid), **kargs )
        
    if ch == 127 or ch == 350 or ch == 190 or ch == 161 or ch == 139:
        v = v[-4:]
        g = g[-4:]
        eg = eg[-4:]
    
    if ch == 21 or ch == 5 or ch == 248 or ch == 52:
        v = v[-3:]
        g = g[-3:]
        eg = eg[-3:]

    print( ch, pmtid, v, g)
    res = dofit( v, g, eg )
        
    #res = dofit( v, g, eg )

    p=res[0]
    ep=res[1]
    xint=res[2]
    yint=res[3]
    vnom=res[4]
    evnom=res[5] 
        
    label= "FIT $a \cdot V^k$: \n"
    label+="a: %.2e $\pm$ %3.e\n" % ( p[0], ep[0] ) 
    label+="b: %.2f $\pm$ %.3f\n" % ( p[1], ep[1] )
    label+='V($4.0 \cdot 10^6$)= %d $\pm$ %d' % ( vnom, evnom )

    ax.plot(xint, yint, color=out[0].get_color(), lw=1.0, label=label)

    ax.set_ylabel("Gain [$10^7$ electrons]")
    ax.set_xlabel("Voltage [V]")
    ax.legend(fontsize=16)
    
    fig.tight_layout()

    return fig#, p[0], p[1]

In [ ]:
recoPMTs = [ 1 , 111, 166, 192, 230, 238, 222, 302, 309, 340, 353 ]
recoChs = [ 350, 248, 190, 161, 139, 127, 131,  59,  52,  21, 5 ]

In [ ]:
def getVoltages(ch):
    
    if ch == 52: #309
        return [1500, 1530, 1550, 1570, 1600, 1650, 1500, 1700, 1800, 1900]
    elif ch == 59: #302
        return [1500, 1530, 1470, 1450, 1430, 1478, 1454, 1478, 1454, 1454]
    elif ch == 131: #222
        return [1500, 1530, 1550, 1470, 1450, 1559, 1531, 1559, 1531, 1531 ]
    
    elif ch == 350: #001
        return [1500, 1530, 1550, 1470, 1450, 1600, 1836, 1700, 1800, 1900]
    elif ch == 190: #166
        return [1500, 1530, 1550, 1470, 1450, 1600, 1881, 1700, 1800, 1900 ]
    elif ch == 161: #192
        return [1500, 1530, 1550, 1470, 1450, 1600, 1950, 1700, 1800, 1900]
    elif ch == 139: #230
        return [1500, 1530, 1550, 1470, 1450, 1600, 1950, 1700, 1800, 1900]
    elif ch == 127: #238
        return [1500, 1530, 1550, 1470, 1450, 1600, 1848, 1700, 1800, 1900 ] 

    return [1500, 1530, 1550, 1470, 1450, 1600, 1500, 1700, 1800, 1900]

In [ ]:
files = [ "backgroundphotons_run11246_1702593081.csv", "backgroundphotons_run11258_1702673047.csv",
          "backgroundphotons_run11260_1702678720.csv", "backgroundphotons_run11262_1702685886.csv",
          "backgroundphotons_run11265_1702694286.csv", "backgroundphotons_run11275_1703010707.csv",
          "backgroundphotons_run11389_1704910035.csv", "backgroundphotons_run11276_1703011354.csv",
          "backgroundphotons_run11327_1703795897.csv", "backgroundphotons_run11328_1703800223.csv" ] 
srcpath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/calibrationdb/"

In [ ]:
def extract_data(channel,files):
    
    v = []
    g = []
    eg = []

    for f in files:
    
        filename = srcpath + f
        runnum = filename.split("_")[1]
        df = pd.read_csv(filename)

        _sel = df.pmt==channel
        
        g.append( df[_sel].q.to_numpy()[0] )
        eg.append( df[_sel]["eq"].to_numpy()[0] )

    v = getVoltages(channel)
    
    return v, g, eg

In [ ]:
pdf = backend_pdf.PdfPages("recovering-PMTs.pdf")


for i, c in enumerate(recoChs):
    
    #print("PMT id {}, ch {}".format(recoPMTs[i],c))
    v, g, eg = extract_data(c,files)
    #fig, a, b, = plotone(c, v, g, eg) 
    fig = plotone(c, recoPMTs[i], v, g, eg) 


    plt.grid(alpha=0.5,linestyle="dashed")
    pdf.savefig( fig )
    plt.show()
    
pdf.close()